アンサンブル用シート

In [9]:
import pandas as pd
import numpy as np

In [10]:
ensemble_result = []
ensemble_result = pd.DataFrame(ensemble_result)

lightGBM_result = pd.read_csv('lightGBM_result.csv')
DART_result = pd.read_csv('DART_result.csv')
#GOSS_result = pd.read_csv('GOSS_result.csv')

#ensemble_result = (lightGBM_result['0'] + DART_result['0'] + GOSS_result['0'])/3
ensemble_result = (lightGBM_result['0'] + DART_result['0'])/2
ensemble_result.reset_index()

,index,0
0,0,136921.316079
1,1,122539.048831
2,2,76669.015018
3,3,61971.030820
4,4,105278.271433
...,...,...
31257,31257,109476.980987
31258,31258,101998.445512
31259,31259,174874.247825
31260,31260,278227.139737


In [11]:
df = pd.read_csv('test.csv')
id = df['id']
result = pd.concat([id, ensemble_result],axis=1)
result.to_csv('result.csv',index=False)

In [12]:
result

,id,0
0,31471,136921.316079
1,31472,122539.048831
2,31473,76669.015018
3,31474,61971.030820
4,31475,105278.271433
...,...,...
31257,62728,109476.980987
31258,62729,101998.445512
31259,62730,174874.247825
31260,62731,278227.139737
